In [87]:
import pandas as pd
import json
import json_lines

In [88]:
### 1. COVID DATA BALI REGENCIES
###  -------------------------------------

## From scrapy spider

In [89]:
## Converting Json-Lines File to pandas DataFrame
##########################################

json_file = r'C:\Users\ansve\Coding\Projects-WebScraping\CovidBali\CovidBali\CovidBali\daily.jl'

##Extract data from First TABLE
###############################################

data_first = []
with open(json_file, 'rb') as f:
    count = 0
    for line in json_lines.reader(f, broken=True):
        count+=1
        if count % 2 == 0: #this is the remainder operator
            for x in range(0, 10):
                data_first.append(line['data'][x])

df_first = pd.DataFrame(data_first)
df_first.to_csv('test.csv')

##Extract data from SECOND TABLE
###############################################
data_second = []

with open(json_file, 'rb') as f:
    count =0
    for line in json_lines.reader(f, broken=True):
        count+= 1
        if count % 2 != 0:
            for x in range(0, 10):
                data_second.append(line['data'][x])
df_second = pd.DataFrame(data_second)
df_second.head()

## Merge both df
#############################
df_merged = pd.merge(df_first, df_second, how= 'left', left_on=[1, 10], right_on=[1, 12])
df_merged.tail(11)
df_bali = df_merged
# df_bali.tail()
df_bali.columns

Index([ '0_x',     10,      1,  '2_x',  '3_x',  '4_x',  '5_x',  '6_x',  '7_x',
        '8_x',  '9_x', '10_x',  '0_y',  '2_y',  '3_y',  '4_y',  '5_y',  '6_y',
        '7_y',  '8_y',  '9_y', '10_y',     11,     12],
      dtype='object')

In [90]:
## Column names from https://infocorona.baliprov.go.id/API/pendataan/laporan-harian-01.php and ...-02.php

columns = [
    'No','Date', 'Kabupaten / County', 'new_treatment', 'total_treatment', 'new_recovered', 'total_recovered', 'new_deaths', 'total_deaths', 'new_cases', 'total_cases', 'Date1', 'no', 'new PPLN/PMI', 'New history-of-foreign-travel/ migrant-worker (PPLN/PMI)', 'new domestic-travel-histor', 'total domestic-travel-history (PPDN)', 'new_local_transmission', 'total_local_transmission',	'new_other_transmission', 'total_other_transmission', 'new_cases' ,'total_cases', 'date' ]


In [91]:
df_bali.columns = columns
# df_bali.head()
df = df_bali.copy()

In [92]:
# delete duplicate columns and convert dtypes
# df = df.loc[:,~df.columns.duplicated()]

## fill NaN in Kabubaten Lain fileds (confirmed cases from outside of bali regency but detected in Bali have no match with reference-data e.g)
################################################
df_merged = df_merged.fillna(0)
df_merged[df_merged.isna().any(axis=1)]

## convert columns
df['No'] = df['No'].astype(int)
df['Name_EN'] = df['Kabupaten / County'].astype(str)
df['Name_EN'] = df['Name_EN'].str.casefold()
df.dtypes

No                                                           int32
Date                                                        object
Kabupaten / County                                          object
new_treatment                                               object
total_treatment                                             object
new_recovered                                               object
total_recovered                                             object
new_deaths                                                  object
total_deaths                                                object
new_cases                                                   object
total_cases                                                 object
Date1                                                       object
no                                                          object
new PPLN/PMI                                                object
New history-of-foreign-travel/ migrant-worker (PPLN/PMI)    ob

In [93]:
from dateutil.parser import parse
df['Date'] = df['date'].str.strip()
df['Date'] = df['Date'].astype(str)
df['Date'] = df['Date'].apply(lambda x: parse(x))
df.Date[1]

Timestamp('2020-12-05 00:00:00')

In [94]:
# data cleaning, processing
#------------------------

# add addtitional data
df_ref = pd.read_excel('Bali_reference_data.xlsx')
df_ref = df_ref.drop(columns=['https://en.wikipedia.org/wiki/Bali#cite_note-BPS2019-2', 'https://sp2010.bps.go.id/index.php/site/tabel?tid=321&wid=0', 'Source'])
df_ref['Name_EN'] = df_ref['Name_Indo'].str.casefold()
df_ref.replace('kota denpasar', 'denpasar', inplace=True)
df_ref.replace('totals', 'bali', inplace=True)
df_ref.replace('Totals', 'Bali', inplace=True)

# df_ref.head(15)

In [99]:
# merge both df on 'Name_EN'
df_merged = df.merge(df_ref, on="Name_EN", how='left')

# convert to integers
int_numbers = ['new_treatment', 'total_treatment', 'new_recovered', 'total_recovered', 'new_deaths', 'total_deaths', 'new_cases', 'total_cases']

df_merged[int_numbers] = df_merged[int_numbers].apply(pd.to_numeric)

# ## add columns
# df_merged['total_cases_per_100k'] = df_merged['total_cases'] / df_merged['population_2015'] * 100000
# df_merged['total_cases_per_100k'] = df_merged['total_cases_per_100k'].apply(pd.to_numeric).round(2)

# df_merged['total_deaths_per_100k'] = df_merged['total_deaths'] / df_merged['population_2015']
# df_merged['total_deaths_per_100k'] = df_merged['total_deaths_per_100k'].apply(pd.to_numeric).round(2)

# df_merged['cases7'] = df_merged['new_cases'].rolling(window=7, center=False).mean()
# df_merged['cases7_per_100k'] = df_merged['cases7'] / df_merged['population_2015']
# df_merged['cases7_per_100k'] = df_merged['cases7_per_100k'].apply(pd.to_numeric).round(2)

# df_merged['total_cases_per_100k'] = df_merged['total_cases'] / df_merged['population_2015']
# df_merged['total_cases_per_100k'] = df_merged['total_cases_per_100k'].apply(pd.to_numeric).round(2)

# df_merged['deaths7'] = df_merged['new_deaths'].rolling(window=7, center=False).mean()
# df_merged['deaths7_per_100k'] = df_merged['total_deaths_per_100k'].rolling(window=7, center=False).mean()
# df_merged['deaths7_per_100k'] = df_merged['deaths7_per_100k'].apply(pd.to_numeric).round(2)

# df_merged[['total_deaths', 'total_cases', 'total_deaths_per_100k', 'population_2015', 'total_cases_per_100k']]

ValueError: Columns must be same length as key

In [39]:
## drop temporary unnecessary columns
drop_columns = ['New PPLN / PMI', 'new PPDN', 'total PPDN',
       'new_local_transmission', 'total_local_transmission',
       'new_other_transmission', 'total_other_transmission', 'Capital', 'population_2000',
       'population_2010', 'HDI_2014_estimate', 'Gouvernor',
       'Nr. of Sub-Districts', 'Nr of Keluruhan/Villages', 'area_km2' ]
df_clean = df_merged.drop(columns=drop_columns)

In [40]:
## check NaN in df 
# nan_rows = df_clean[df_clean['population_2015'].isnull()]
# nan_rows

In [41]:
# infection fatality ratio (IFR), which estimates this proportion of deaths among all infected individuals. The second is case fatality ratio (CFR), which estimates this proportion of deaths among identified confirmed cases. 
# see https://www.who.int/news-room/commentaries/detail/estimating-mortality-from-covid-19

df_merged['CFR'] = df_merged['total_deaths'] / df_merged['total_cases'] * 100
df_merged['CFR'] = df_merged['CFR'].round(2)
df_merged['growth_rate'] = df_merged['cases7'].pct_change(fill_method ='ffill', periods=7)

In [42]:
## add id from geojson file
geojson_bali = 'new_bali_id.geojson'

# open new geojson (with id) file
bali_geo_id = json.load(open(geojson_bali, 'r'))

## get names of regencies
regencies = []
for regency in bali_geo_id['features']:
  regencies.append(regency['properties']['ADM2_EN'])

  # add id column in df with feature.id from geojson
bali_id_map = {}
for regency in bali_geo_id['features']:
  bali_id_map[regency['properties']['ADM2_EN']] = regency['id']
bali_id_map

## for Foreigners or Other Regency add 'id' = 0
df_merged['id'] = df_merged['Name_Indo'].apply(lambda x: bali_id_map[x] if x in bali_id_map else 0)
df_merged.head()

,No,Kabupaten / County,New PPLN / PMI,new PPDN,total PPDN,new_local_transmission,total_local_transmission,new_other_transmission,total_other_transmission,new_cases,...,new_total,total_cases_per_100k,total_deaths_per_100k,cases7,cases7_per_100k,deaths7,deaths7_per_100k,CFR,growth_rate,id
0,1,JEMBRANA,0,0,11,1,626,0,0,1,...,1,2.44,0.05,NaN,NaN,NaN,NaN,2.11,NaN,5
1,2,TABANAN,0,2,10,23,1466,0,0,25,...,25,3.44,0.11,NaN,NaN,NaN,NaN,3.14,NaN,9
2,3,BADUNG,0,0,4,18,2478,0,0,18,...,18,4.08,0.08,NaN,NaN,NaN,NaN,2.03,NaN,1
3,4,DENPASAR,0,-1,22,28,3891,0,0,27,...,27,4.51,0.10,NaN,NaN,NaN,NaN,2.27,NaN,8
4,5,GIANYAR,0,-1,10,16,1951,0,0,15,...,15,4.02,0.15,NaN,NaN,NaN,NaN,3.77,NaN,4


In [43]:
df_merged.to_csv('bali_regency_data.csv')

In [44]:
### 2. INDONESIA COVID DATA 
###  -------------------------------------

## From Kaggel Dataset
# 1. connect to kaggle and get latest dataset


In [45]:
pip install kaggle

Note: you may need to restart the kernel to use updated packages.


In [46]:
from kaggle.api.kaggle_api_extended import KaggleApi

# from keys import kaggle_api

## 1. with Kaggle Api
api =KaggleApi()
api.authenticate()

# downoad single file
#Signature: dataset_download_file(dataset, file_name, path=None, force=False, quiet=True)
api.dataset_download_files('hendratno/covid19-indonesia/covid_19_indonesia_time_series_all.csv', unzip = True)
# 

In [47]:
df_indo = pd.read_csv('covid_19_indonesia_time_series_all.csv')

df_indo.head()

,Date,Location ISO Code,Location,New Cases,New Deaths,New Recovered,New Active Cases,Total Cases,Total Deaths,Total Recovered,...,Longitude,Latitude,New Cases per Million,Total Cases per Million,New Deaths per Million,Total Deaths per Million,Case Fatality Rate,Case Recovered Rate,Growth Factor of New Cases,Growth Factor of New Deaths
0,3/1/2020,ID-JK,DKI Jakarta,2,0,0,2,489,20,39,...,106.836118,-6.204699,0.18,45.09,0.0,1.84,4.09%,7.98%,NaN,NaN
1,3/2/2020,ID-JK,DKI Jakarta,2,0,0,2,491,20,39,...,106.836118,-6.204699,0.18,45.27,0.0,1.84,4.07%,7.94%,1.0,1.0
2,3/2/2020,IDN,Indonesia,2,0,0,2,2,0,0,...,113.921327,-0.789275,0.01,0.01,0.0,0.00,0.00%,0.00%,NaN,NaN
3,3/2/2020,ID-JB,Jawa Barat,3,0,0,3,12,5,120,...,107.603708,-6.920432,0.07,0.27,0.0,0.11,41.67%,1000.00%,NaN,NaN
4,3/2/2020,ID-RI,Riau,1,0,0,1,2,1,1,...,101.805109,0.511648,0.16,0.33,0.0,0.16,50.00%,50.00%,NaN,NaN


In [48]:
# df_indo.dtypes

In [49]:
print(list(df_merged.columns))

['No', 'Kabupaten / County', 'New PPLN / PMI', 'new PPDN', 'total PPDN', 'new_local_transmission', 'total_local_transmission', 'new_other_transmission', 'total_other_transmission', 'new_cases', 'total_cases', 'date', 'new_treatment', 'total_treatment', 'new_recovered', 'total_recovered', 'new_deaths', 'total_deaths', 'Name_EN', 'Date', 'Name_Indo', 'Capital', 'area_km2', 'population_2000', 'population_2010', 'population_2015', 'HDI_2014_estimate', 'Gouvernor', 'Nr. of Sub-Districts', 'Nr of Keluruhan/Villages', 'new_total', 'total_cases_per_100k', 'total_deaths_per_100k', 'cases7', 'cases7_per_100k', 'deaths7', 'deaths7_per_100k', 'CFR', 'growth_rate', 'id']


In [50]:
# adjust column names
columns_new = ['Date', 'Location ISO Code', 'Location', 'new_cases', 'new_deaths',
       'new_recovered', 'New Active Cases', 'total_cases', 'total_deaths',
       'total_recovered', 'Total Active Cases', 'Location Level',
       'City or Regency', 'Province', 'Country', 'Continent', 'Island',
       'Time Zone', 'Special Status', 'Total Regencies', 'Total Cities',
       'Total Districts', 'Total Urban Villages', 'Total Rural Villages',
       'Area (km2)', 'Population', 'Population Density', 'Longitude',
       'Latitude', 'New Cases per Million', 'Total Cases per Million',
       'New Deaths per Million', 'Total Deaths per Million',
       'CFR', 'Case Recovered Rate',
       'Growth Factor of New Cases', 'Growth Factor of New Deaths']

df_indo.columns = columns_new
df_indo.columns

Index(['Date', 'Location ISO Code', 'Location', 'new_cases', 'new_deaths',
       'new_recovered', 'New Active Cases', 'total_cases', 'total_deaths',
       'total_recovered', 'Total Active Cases', 'Location Level',
       'City or Regency', 'Province', 'Country', 'Continent', 'Island',
       'Time Zone', 'Special Status', 'Total Regencies', 'Total Cities',
       'Total Districts', 'Total Urban Villages', 'Total Rural Villages',
       'Area (km2)', 'Population', 'Population Density', 'Longitude',
       'Latitude', 'New Cases per Million', 'Total Cases per Million',
       'New Deaths per Million', 'Total Deaths per Million', 'CFR',
       'Case Recovered Rate', 'Growth Factor of New Cases',
       'Growth Factor of New Deaths'],
      dtype='object')

In [71]:

# add columns so consistent with Bali_regency data
df_indo['Name_EN'] = df_indo['Location'].str.casefold()

df_indo['new_cases_per_100k'] = df_indo['New Cases per Million'] / 100.0
df_indo['new_cases_per_100k'] = df_indo['new_cases_per_100k'].round(2)

df_indo['new_deaths_per_100k'] = df_indo['New Deaths per Million'] / 100.0
df_indo['new_deaths_per_100k']= df_indo['new_deaths_per_100k'].round(2)

df_indo['total_cases_per_100k'] = df_indo['Total Cases per Million'] / 100.0
df_indo['total_cases_per_100k'] = df_indo['total_cases_per_100k'].round(2)

df_indo['total_deaths_per_100k'] = df_indo['Total Deaths per Million'] / 100.0
df_indo['total_deaths_per_100k'] = df_indo['total_deaths_per_100k'].round(2)

# rolling 7-day average
df_indo['cases7'] = df_indo['new_cases'].rolling(window=7, center=False).mean().round(2)
df_indo['deaths7'] = df_indo['new_deaths'].rolling(window=7, center = False).mean().round(2)

df_indo['cases7_per_100k'] = df_indo['cases7'] / df_indo['Population'] *100000
df_indo['deaths7_per_100k'] = df_indo['deaths7'] / df_indo['Population'] *100000
df_indo['cases7_per_100k'] = df_indo['cases7_per_100k'].round(2)
df_indo['deaths7_per_100k'] = df_indo['deaths7_per_100k'].round(2)

df_indo['growth_rate'] = df_indo['Growth Factor of New Cases']
# df_indo['growth_rate'] = df_indo['cases7'].pct_change(periods=7, fill_method='ffill')

df_indo.tail()

,Date,Location ISO Code,Location,new_cases,new_deaths,new_recovered,New Active Cases,total_cases,total_deaths,total_recovered,...,new_cases_per_100k,new_deaths_per_100k,total_cases_per_100k,total_deaths_per_100k,cases7,deaths7,cases7_per_100k,deaths7_per_100k,growth_rate,id
9954,12/31/2020,ID-SG,Sulawesi Tenggara,38,0,24,14,7907,147,6696,...,0.1442,0.0000,30.00,0.56,133.86,0.29,5.08,0.01,0.60,23
9955,12/31/2020,ID-SA,Sulawesi Utara,88,1,0,87,9671,310,7003,...,0.3331,0.0038,36.61,1.17,138.57,0.29,5.25,0.01,2.59,22
9956,12/31/2020,ID-SB,Sumatera Barat,117,3,26,88,23464,522,18030,...,0.2120,0.0054,42.51,0.95,150.57,0.71,2.73,0.01,0.94,2
9957,12/31/2020,ID-SS,Sumatera Selatan,92,1,50,41,11826,599,9364,...,0.1120,0.0012,14.39,0.73,148.00,0.71,1.80,0.01,0.84,33
9958,12/31/2020,ID-SU,Sumatera Utara,83,0,82,1,18149,679,15403,...,0.0558,0.0000,12.20,0.46,156.00,0.71,1.05,0.00,0.93,4


In [52]:
## add id from geojson
geojson_indo = 'new_indo_id.geojson'

# open new geojson (with id) file
indo_geo_id = json.load(open(geojson_indo, 'r'))


In [53]:
indo_geo_id['features'][7]['id']

8

In [54]:
## ADJUST NAMES OF df_indo with NAMES OF GEOJSON ADM1_EN

# which names are different
names_df = df_indo['Location'].unique()
provinces = []
for province in indo_geo_id['features']:
  provinces.append(province['properties']['state'])

list(set(provinces) - set(names_df))

['Yogyakarta', 'Jakarta Raya', 'Bangka-Belitung', 'Irian Jaya Barat']

In [55]:
list_df = ['DKI Jakarta', 'Kalimantan Utara', 'Daerah Istimewa Yogyakarta', 'Papua Barat','Kepulauan Bangka Belitung']

list(set(provinces) - set(names_df))
list1= ['Bangka-Belitung', 'Jakarta Raya', 'Yogyakarta', 'Irian Jaya Barat']


In [56]:
# adjust Names of provnices in Indo

df_indo = df_indo.replace("DKI Jakarta", 'Jakarta Raya')
df_indo = df_indo.replace("Papua Barat", 'Irian Jaya Barat')
df_indo = df_indo.replace("Daerah Istimewa Yogyakarta", 'Yogyakarta')
df_indo = df_indo.replace("Kepulauan Bangka Belitung", 'Bangka-Belitung')

In [57]:
provinces = []
for province in indo_geo_id['features']:
  provinces.append(province['properties']['state'])

# add id column in df with feature.id from geojson
indo_id_map = {}
for province in indo_geo_id['features']:
  indo_id_map[province['properties']['state']] = province['id']
# indo_id_map

# ## for Foreigners or Other Regency add 'id' = 0
df_indo['id'] = df_indo['Location'].apply(lambda x: indo_id_map[x] if x in indo_id_map else 0)

df_indo.head()


,Date,Location ISO Code,Location,new_cases,new_deaths,new_recovered,New Active Cases,total_cases,total_deaths,total_recovered,...,new_cases_per_100k,new_deaths_per_100k,total_cases_per_100k,total_deaths_per_100k,cases7,deaths7,cases7_per_100k,deaths7_per_100k,growth_rate,id
0,3/1/2020,ID-JK,Jakarta Raya,2,0,0,2,489,20,39,...,0.0018,0.0,0.4509,0.0184,NaN,NaN,NaN,NaN,NaN,27
1,3/2/2020,ID-JK,Jakarta Raya,2,0,0,2,491,20,39,...,0.0018,0.0,0.4527,0.0184,NaN,NaN,NaN,NaN,NaN,27
2,3/2/2020,IDN,Indonesia,2,0,0,2,2,0,0,...,0.0001,0.0,0.0001,0.0000,NaN,NaN,NaN,NaN,NaN,0
3,3/2/2020,ID-JB,Jawa Barat,3,0,0,3,12,5,120,...,0.0007,0.0,0.0027,0.0011,NaN,NaN,NaN,NaN,NaN,29
4,3/2/2020,ID-RI,Riau,1,0,0,1,2,1,1,...,0.0016,0.0,0.0033,0.0016,NaN,NaN,NaN,NaN,NaN,18


In [58]:
df_indo.to_csv('indo_province_data.csv')

In [59]:
### 3. World Data
##------------------------


from kaggle.api.kaggle_api_extended import KaggleApi

# from keys import kaggle_api

## 1. with Kaggle Api
api =KaggleApi()
api.authenticate()

# downoad single file
#Signature: dataset_download_file(dataset, file_name, path=None, force=False, quiet=True)
# api.dataset_download_files('imdevskp/corona-virus-report/worldometer_data.csv', unzip=True)

# ## Datasets

# REALLY GOOD !!!!
# https://github.com/owid/covid-19-data/blob/master/public/data/owid-covid-data.csv
